<a href="https://colab.research.google.com/github/ThassiAmorim/Pdf2Excel/blob/main/Pdf2Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF openpyxl pytesseract pillow
!apt install libtesseract-dev
!sudo apt install tesseract-ocr
!sudo apt-get install tesseract-ocr-por
import fitz  # PyMuPDF
import openpyxl
import pytesseract
from PIL import Image
import os
import cv2




In [ ]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Caminho para o arquivo PDF
caminho_pdf = '/content/drive/MyDrive/AutomatizaNotas/notas.pdf'

In [ ]:
from difflib import SequenceMatcher

def similaridade(str1, str2):
    # Inicializar o SequenceMatcher com duas strings
    sequence_matcher = SequenceMatcher(None, str1, str2)

    # Mapear a similaridade para uma escala de 0 a 10
    score = round(sequence_matcher.ratio() * 10, 2)

    return score

# Função para extrair texto de uma imagem usando OCR
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    print(text)
    return text

# Função para extrair e converter imagens em texto
def fotos_para_texto(page):
    text = ""
    foto = page.get_images(full=True)

    for img_index, img in enumerate(foto):
        xref = img[0]
        base_image = pdf_document.extract_image(xref)
        image_bytes = base_image["image"]

        # Salvar a imagem em um arquivo temporário para que o pytesseract possa lê-la
        temporario = f"temp_image_{img_index}.png"
        with open(temporario, "wb") as image_file:
            image_file.write(image_bytes)

        # Usar pytesseract para converter a imagem em texto
        extracted_text = pytesseract.image_to_string(temporario, lang='por')

        # Adicionar o texto extraído à saída
        text += f"Imagem {img_index + 1} (Página {page_num + 1}):\n{extracted_text}\n\n"

        # Remover o arquivo de imagem temporário
        image_file.close()

    return text

# Função para extrair dados do texto
def raspagem_dados(text):
    lines = text.split('\n') #[line.strip() for line in text.split('\n')]
    estabelecimento = ""
    total = ""

    for line in lines:
        #print(f"-------------{line}---------------------")
        if "Estabelecimento:" in line:
            estabelecimento = line.split("Estabelecimento:")[1].strip()
            partes = estabelecimento.split("EF", 1)
            estabelecimento = partes[0]
        if similaridade("VALOR TOTAL DA GUIA DE ENTREGA R$ ", line) > 6:
             try:
                # Tentar dividir a linha por "R$" ou "RS"
                total = line.split("R$")[1].strip()
             except IndexError:
                try:
                    total = line.split("RS")[1].strip()
                except IndexError:
                    # Se ocorrer um erro ao dividir a linha, atribuir 0 ao total
                    total = 0
             if total and (total[-1] == ']' or total[-1] == ')' or total[-1] == '|'):
                total = total[:-1]

             print(f"---Estabelecimento:{estabelecimento} ----Total:{total}")

             return estabelecimento, total

# Abrir o PDF com PyMuPDF
pdf_document = fitz.open(caminho_pdf)

# Modificar arquivo Excel
excel_path = '/content/drive/MyDrive/AutomatizaNotas/extractions.xlsx'
workbook = openpyxl.Workbook()
worksheet = workbook.active
worksheet.title = "Extrações"
worksheet.append(["estabelecimento", "total"])

# Extrair dados das imagens e adicionar à planilha Excel
for page_num in range(pdf_document.page_count):
    page = pdf_document[page_num]

    estabelecimento, total = raspagem_dados(fotos_para_texto(page))
    worksheet.append([estabelecimento, total])

# Salvar o arquivo Excel
workbook.save(excel_path)

# Fechar o documento PDF
#pdf_document.close()

print("Extração e criação do Excel concluídas!")